In [3]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import time

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

C:\Users\harsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
X = mnist.data
Y = mnist.target
print('number of examples :', X.shape[0])

number of examples : 70000


In [5]:
x_time = time.time()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 40, test_size = 0.14)
clf = GaussianNB()
# clf.fit(X_train, Y_train)
y_time = time.time()
print(y_time-x_time)

0.3440134525299072


In [11]:
x_time = time.time()
param_grid = {
    'priors' : [ [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],  None],
    'var_smoothing' : np.arange( 0.1, 1, 0.01)
}

random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=20, cv=5, random_state=42, n_jobs=2 )
random_search.fit(X_train, Y_train)

y_time = time.time()
print("Time required(in sec)", y_time-x_time)

print('Best hyperparameters:', random_search.best_params_)
print('Best score:', random_search.best_score_)

Time required(in sec) 119.1869568824768
Best hyperparameters: {'var_smoothing': 0.13, 'priors': None}
Best score: 0.8010764061650135


In [7]:
clf_best = GaussianNB(**random_search.best_params_)
clf_best.fit(X_train, Y_train)

GaussianNB(var_smoothing=0.13)

In [9]:
x_time = time.time()
Y_pred_train = clf_best.predict( X_train )
Y_pred_test = clf_best.predict( X_test )
y_time = time.time()
print("Time required (in sec)", y_time-x_time)
print('Accuracy Score on train data: ', accuracy_score(y_true=Y_train, y_pred=Y_pred_train))
print('Accuracy Score on test data: ', accuracy_score(y_true=Y_test, y_pred=Y_pred_test))

Time required (in sec) 4.1944122314453125
Accuracy Score on train data:  0.8015913885612718
Accuracy Score on test data:  0.8050198959289868


In [10]:
print("For the Test Cases : ")
f1score = f1_score(Y_test, Y_pred_test, average='macro')
print("F1 Score: ", f1score)
cm = confusion_matrix(Y_test, Y_pred_test)
print("Confusion Matrix:")
print(cm)

For the Test Cases : 
F1 Score:  0.8034425938608944
Confusion Matrix:
[[ 865    1    5    4    2   14   21    0   58    6]
 [   0 1049    2    2    0    3    4    1   15    2]
 [  13   47  770   23   18    5   55   12   56    6]
 [  10   59   35  788    1   20   14   11   61   40]
 [   5   16    6    0  669    2   19    4   23  222]
 [  21   34   10  101   24  535   22    4   54   47]
 [   3   31    3    0    5   12  870    0   10    1]
 [   6   33    1    2   26    0    0  857   13   89]
 [   8  109    8   48   17   25    5    4  687   58]
 [   8   34    3    8   51    0    1   21   28  800]]
